In [186]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [187]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [200]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [201]:
%%capture pwd
!pwd

In [202]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-is-the-cost-of-living-crisis-affecting-retailers-and-their-customers',
 300,
 500)

# Fig 4

In [203]:
df = pd.read_csv("raw/Dataset 0.csv",header=None)
df['time']=pd.to_datetime('2006-01-01')+pd.to_timedelta('1y')/4*(df[0]-1.05)
df=df.rename(columns={1:'value'}).drop(0,axis=1)

/opt/conda/lib/python3.9/site-packages/pandas/core/tools/timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


In [204]:
f = "fig4_index"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,value,time
0,-3.387910,2006-01-01 00:36:40.088009914
1,-3.631362,2006-01-13 04:55:21.009142108
2,-3.917338,2006-01-18 17:47:28.700565827
3,-4.330397,2006-01-27 14:22:53.006843784
4,-4.553006,2006-01-23 04:05:10.853704806


In [205]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "time:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=['2005-11-01', '2022-08-01']),
    )
)
line = base.mark_line(color=colors["eco-mid-blue"],interpolate="basis").encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="UK Index June 2022",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
dashes=base.mark_line(
    color=colors["eco-turquiose"],interpolate="basis",strokeDash=[5, 5]
).transform_window(
    rolling_mean='mean(value)',
    frame=[-95, 0]
).encode(
    y='rolling_mean:Q'
)
# top = line.mark_point(
#     size=25, shape="line", color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
# ).encode(y="e2:Q")
# area = base.mark_errorbar(
#     opacity=0.4, color=colors["eco-turquiose"], ticks=True
# ).encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
axis = (
    alt.Chart(pd.DataFrame([{"x": "2005-11-01", "y": 0}, {"x": "2022-08-01", "y": 0}]))
    .mark_line(strokeWidth=1, opacity=0.4,color=colors["eco-gray"])
    .encode(x=alt.X("x:T", sort=[]), y="y:Q")
)
label1 = (
    alt.Chart(pd.DataFrame([{"x": "2022-08-01", "y": -40,'t':'Index'}]))
    .mark_text(color=colors["eco-mid-blue"],align='left',dy=5)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q",text='t:N')
)
label2 = (
    alt.Chart(pd.DataFrame([{"x": "2022-08-01", "y": -22,'t':'Annual moving average'}]))
    .mark_text(color=colors["eco-turquiose"],align='left',dy=5)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q",text='t:N')
)
layer1 = (
    ((axis+line+dashes+label1+label2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

In [206]:
df = pd.read_excel("raw/data.xlsx", skiprows=43, nrows=8, usecols="A:C")
df.columns = ["x", "y", "e"]
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]

FileNotFoundError: [Errno 2] No such file or directory: 'raw/data.xlsx'

In [207]:
f = "fig5_margin"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,value,time
0,-3.387910,2006-01-01 00:36:40.088009914
1,-3.631362,2006-01-13 04:55:21.009142108
2,-3.917338,2006-01-18 17:47:28.700565827
3,-4.330397,2006-01-27 14:22:53.006843784
4,-4.553006,2006-01-23 04:05:10.853704806


In [208]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=-20,
            labelOffset=10,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
top = line.mark_point(
    size=25, shape="line", color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
).encode(y="e2:Q")
area = base.mark_errorbar(
    opacity=0.4, color=colors["eco-turquiose"], ticks=True
).encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "Interceptions", "y": 0}, {"x": "Fulltime", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((top + area + line + points + axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 6

In [209]:
df = pd.read_csv("raw/wpd_datasets.csv",header=None,skiprows=2)
df['time1']=pd.to_datetime('2015-01-01')+pd.to_timedelta('1y')/3*(df[0]-0.9)
df=df.rename(columns={1:'value1'}).drop(0,axis=1)
df['time2']=pd.to_datetime('2015-01-01')+pd.to_timedelta('1y')/3*(df[2]-0.9)
df=df.rename(columns={3:'value2'}).drop(2,axis=1)
df['time3']=pd.to_datetime('2015-01-01')+pd.to_timedelta('1y')/3*(df[4]-0.9)
df=df.rename(columns={5:'value3'}).drop(4,axis=1)

/opt/conda/lib/python3.9/site-packages/pandas/core/tools/timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/opt/conda/lib/python3.9/site-packages/pandas/core/tools/timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/opt/conda/lib/python3.9/site-packages/pandas/core/tools/timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


In [210]:
f = "fig6_injuries"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,value1,value2,value3,time1,time2,time3
0,-2.603407,0.233291,-2.855104,2015-01-01 13:08:14.214620687,2015-01-03 21:55:58.228398768,2015-01-03 19:20:02.507282375
1,-3.286932,0.176618,-3.027078,2015-01-12 18:23:38.264132184,2015-01-13 13:40:46.019974438,2015-01-09 20:20:18.343595255
2,-3.532604,0.074580,-3.111242,2015-01-21 10:57:24.712931519,2015-01-21 00:07:17.858817264,2015-01-18 04:29:56.591345903
3,-3.723814,-0.003734,-3.286130,2015-01-26 16:13:34.289928983,2015-01-31 10:35:42.271165248,2015-01-22 04:01:48.261974807
4,-3.827263,-0.012713,-3.345646,2015-02-02 14:47:48.822171792,2015-02-09 19:24:44.292184919,2015-01-30 14:05:45.402723671


In [211]:
base = alt.Chart(f6).encode(
    x=alt.X(
        "time1:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
)
line1 = base.mark_line(color=colors['eco-dot'],interpolate="basis").encode(
    y=alt.Y(
        "value1:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Inflation [%]",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
).transform_filter("toDate(datum.time1)<toDate('2022-04-01')")
line2 = base.mark_line(color=colors['eco-turquiose'],interpolate="basis").encode(
    x='time2:T',
    y=alt.Y(
        "value2:Q",
        sort=[],
    )
).transform_filter("toDate(datum.time2)<toDate('2022-04-01')")
line3 = base.mark_line(color=colors['eco-mid-blue'],interpolate="basis").encode(
    x='time3:T',
    y=alt.Y(
        "value3:Q",
        sort=[],
    )
).transform_filter("toDate(datum.time3)<toDate('2022-04-01')")
axis = (
    alt.Chart(pd.DataFrame([{"x": "2014-12-01", "y": 0}, {"x": "2022-08-01", "y": 0}]))
    .mark_line(strokeWidth=1, opacity=0.4,color=colors["eco-gray"])
    .encode(x=alt.X("x:T", sort=[]), y="y:Q")
)
label1 = (
    alt.Chart(pd.DataFrame([{"x": "2022-04-01", "y": 32,'t':'Agflation'}]))
    .mark_text(color=colors["eco-dot"],align='left',dy=5)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q",text='t:N')
)
label2 = (
    alt.Chart(pd.DataFrame([{"x": "2022-04-01", "y": 9,'t':'CPI'}]))
    .mark_text(color=colors["eco-turquiose"],align='left',dy=5)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q",text='t:N')
)
label3 = (
    alt.Chart(pd.DataFrame([{"x": "2022-04-01", "y": 6.5,'t':'CPI Food'}]))
    .mark_text(color=colors["eco-mid-blue"],align='left',dy=5)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q",text='t:N')
)
layer1 = (
    (
        (axis+line1 + line2 +line3+label1+label2+label3).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)